In [1]:
##Load all the required libraries...

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy.optimize import curve_fit
import time
import pandas as pd

from tools.base import *
from tools.ddmtd import ddmtd
from time import sleep

In [2]:
server="pi@192.168.222.5" # your ip address might be different

## IMPORTANT##
## make sure you have your ssh keys set-up properly
## END IMPORTANT##
## Contact rohith.saradhy@cern.ch for password to the RPi

In [4]:
#Copy Flash_Firmware scripts to the Raspberry Pi.
!rsync -ra ../rpi_side/Flash_Firmware {server}:

In [6]:
## Flash The Firmware to the Nexys Board flash memory
## Physically restart the Nexys Board after the flash. Or click the "PROG" button on Nexys Video Board.

!../rpi_side/Flash_Nex.sh ../rpi_side/HEX_Files/2022_12_03_v2_031.hex {server}   # clocked at neg edge

#If "chip ID incorrect" error occurs, repeat the process

# Successfull Flashing would look similar to the following:
# We are in dir::/home/pi/Flash_Firmware

# 	FLASH MEMORY WRITE

# Current Chip ID: 010219

# Reading hex file:	 Success
# Number of Bytes: 	 2161808
# Erasing sectors: 	 Success 
# Flashing memory: 	 Success 


# The flash memory was successfully written.

2022_12_03_v2_031.hex                         100% 3324KB  23.9MB/s   00:00    
We are in dir::/home/pi/Flash_Firmware

	FLASH MEMORY WRITE

Current Chip ID: 010219

Reading hex file:	 Success
Number of Bytes: 	 1701924
Erasing sectors: 	 Success 
Flashing memory: 	 Success 

The flash memory was successfully written.


In [8]:
# restart FPGA & check firmware
sleep(1)
!../rpi_side/runAtNex.sh ../restart_FPGA.sh 0 0 {server}
sleep(1)
#Check Version Number
!rsync -ra ../rpi_side/Flash_Firmware {server}:
!../rpi_side/runAtNex.sh bin/check_firmware.exe 0 1 {server} #flash the configuration file
# Should return: 
# FirmwareVersion::2.030 at least

Scipt Location::/home/rsaradhy/work/nexys_ddmtd_github/rpi_side
Scipt Location::/home/rsaradhy/work/nexys_ddmtd_github/rpi_side
RESET DONE 
HEATING 1 STATUS: 0
HEATING 2 STATUS: 0
Firmware Version: 2.031 


In [9]:
# Setting all the variables...
N=10000  #Setting the DDMTD N
freq="40.079"#in MHz #Setting the input clock frequency
pll_config_folder="../rpi_side/PLL_Conf/bcp_configs/"
fig_save_folder = f"./bcp_test/figures/N{N}/"
data_save_folder = f"./bcp_test/data/N{N}/"
!mkdir -p {fig_save_folder} #create those directories
!mkdir -p {data_save_folder} #create those directories
run_number = 1
run_name   = f"bcp_testRun{run_number}"

## PLL Configs

In [10]:
## Availble Configurations...
!ls -t {pll_config_folder}

bcp_nexys_1000001_40.079MHz.h  bcp_nexys_100001_40.079MHz.h
bcp_nexys_10001_40.079MHz.h    bcp_nexys_conf.h


In [11]:
## Configuring the PLL. This needs to be done everytime the board is restarted/ loses power.
##Selecting the Register File##

pll_config = f"{pll_config_folder}bcp_nexys_{N+1}_{freq}MHz.h" #Selecting the configuration according to N, freq
print("Using PLL Config: \n ",pll_config)
print("\n\n")
!scp {pll_config} {server}:Flash_Firmware/include/Si5344_REG.h #Copy the config to the RPi as Si5344
## Compile and Running configuration script
!../rpi_side/runAtNex.sh bin/ddmtd_pll.exe 1 1 {server} #flash the configuration file
#The output should look like the following::
# Done Compiling MEM
# Done Compiling PLL
# address = 0x0514, value = 0x01 ; Value returned = 0x00   
# address = 0x001c, value = 0x01 ; Value returned = 0x00 

# After configuration, D304 will blink every 500 cycles of the beat clock

Using PLL Config: 
  ../rpi_side/PLL_Conf/bcp_configs/bcp_nexys_10001_40.079MHz.h



bcp_nexys_10001_40.079MHz.h                   100%   45KB  12.6MB/s   00:00    
Scipt Location::/home/rsaradhy/work/nexys_ddmtd_github/rpi_side
Done Compiling data_acq
Done Compiling PLL
Done Compiling CheckFirmware
address = 0x0514, value = 0x01 ; Value returned = 0xff   
address = 0x001c, value = 0x01 ; Value returned = 0xff   


## Data Acquisition

In [ ]:
!../rpi_side/runAtNex.sh bin/data_acq.exe 1 2 {server}
# Copy over the files...
!scp {server}:Flash_Firmware/data/ddmtd1.txt {data_save_folder+run_name}ddmtd1.txt
!scp {server}:Flash_Firmware/data/ddmtd2.txt {data_save_folder+run_name}ddmtd2.txt 

Scipt Location::/home/rsaradhy/work/nexys_ddmtd_github/rpi_side
Done Compiling data_acq
Done Compiling PLL
Done Compiling CheckFirmware
